# Statistical analysis using DESeq2

## Installation

In [1]:
library("DESeq2")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The following objects are maske

## Functionality

In [2]:
doy <- function(samples){
    doys <- rep(1,length(samples))
    i = 1
    for (sample in samples){
        x = as.Date(sample,"%y%m%d")
        day = as.numeric(strftime(x, format = "%j"))
        doys[i] = day
        i = i+1
    }
    doys
}

In [3]:
dseq_groups <- function(coldata,countdata,group1,group2){
    coldata = as.data.frame(coldata)
    this_coldata = coldata[coldata$group%in%c(group1,group2),]
    
    this_coldata$group = droplevels(this_coldata$group)
    dseq = DESeqDataSetFromMatrix(countData = countdata[,rownames(this_coldata)],colData = this_coldata, design = ~group)
    dseq_run = DESeq(dseq)
    res = results(dseq_run)
    string=paste(group2,group1,sep=" vs ")
    res$groups = rep(string,nrow(res))
    res$transporter = row.names(res)
    res
}

## Read filtered counts data for abundant transporters

Because the power of statistical analyses of metagenomes depends heavily on the number of reads mapped to features, only TIGRFAMs with >= 100 average counts across datasets were included here. The representative TIGRFAMs of each transporter was used as a proxy for that transporter and the read counts summed.

In [4]:
mg_counts <- read.table("results/mg/rep_trans_filt.raw_counts.tsv", header=T, sep="\t", row.names=1)
colnames(mg_counts) <- gsub("X","",colnames(mg_counts))

In [5]:
head(mg_counts)

,120314,120322,120328,120403,120416,120419,120423,120516,120531,120604,⋯,120920,120924,121001,121004,121015,121022,121028,121105,121128,121220
T534,382,883,204,131,168,443,90,69,188,90,⋯,836,1464,604,177,624,240,138,168,326,1003
T37,362,1826,192,130,158,458,72,83,237,77,⋯,928,944,535,138,704,352,164,135,373,984
T42,181,652,79,59,51,148,34,31,89,40,⋯,553,542,354,73,430,158,95,68,163,449
T7,935,2419,668,424,459,1182,226,141,597,157,⋯,3753,3881,2762,653,2781,1280,645,583,1484,4176
T71,403,2023,567,363,417,1294,263,207,684,152,⋯,2154,2484,1662,364,1682,748,303,391,828,2842
T56,930,1897,561,268,237,939,131,154,765,274,⋯,1824,2744,1272,314,1363,512,258,341,646,1584


In [6]:
mt_counts <- read.table("results/mt/rep_trans_filt.raw_counts.tsv", header=T, sep="\t", row.names=1)
colnames(mt_counts) <- gsub("X","",colnames(mt_counts))

In [7]:
head(mt_counts)

,120516,120613,120712,120813,120927,121024,121220,130123,130226,130403,⋯,130815,130905,131003,140408,140506,140604,140709,140820,140916,141013
T37,98,131,93,208,126,137,270,351,138,69,⋯,34,56,79,11,16,66,162,154,59,69
T7,538,120,813,448,609,776,629,1537,864,1478,⋯,351,331,338,840,601,239,678,719,346,565
T71,447,177,684,403,333,401,295,568,295,193,⋯,241,557,409,226,203,338,542,1014,283,473
T56,366,67,563,120,215,204,191,485,380,512,⋯,107,157,163,860,2229,159,589,191,239,474
T18,1324,1906,297,102,774,376,230,465,601,693,⋯,18,34,53,160,113,168,26,102,76,201
T5,493,416,562,388,688,753,766,1353,629,686,⋯,272,333,227,252,48,295,1431,985,478,393


In [25]:
rownames(mt_counts)

[1] "T37"   "T7"    "T71"   "T56"   "T18"   "T5"    "T67"   "T40"   "T388" 
[10] "T761"  "T12"   "T35"   "T27"   "T724"  "T60"   "T1059" "T59"   "T133" 
[19] "T154"  "T109"  "T3"    "T317"

## Set sample metadata

### Read group information on samples

In [8]:
mg_groups <- read.table("results/mg/samplegroups.tab", header=T, sep="\t", col.names = c("Sample","Group"))
rownames(mg_groups) = mg_groups$Sample
mg_groups[order(mg_groups$Group),]

,Sample,Group
120314,120314,early-spring
120322,120322,early-spring
120531,120531,early-summer
120604,120604,early-summer
120613,120613,early-summer
120619,120619,early-summer
120628,120628,early-summer
120705,120705,fall
120820,120820,fall
120823,120823,fall


In [9]:
mt_groups <- read.table("results/mt/samplegroups.tab", header=T, sep="\t", col.names = c("Sample","Group"))
rownames(mt_groups) = mt_groups$Sample
mt_groups[order(mt_groups$Group),]

,Sample,Group
130605,130605,early-summer
140408,140408,early-summer
140506,140506,early-summer
120712,120712,spring
130403,130403,spring
130416,130416,spring
130422,130422,spring
130507,130507,spring
140604,140604,spring
120813,120813,summer


Use only samples that are clustered into groups.

In [10]:
mg_counts <- mg_counts[as.character(mg_groups$Sample)]

In [11]:
mt_counts <- mt_counts[as.character(mt_groups$Sample)]

### Create dataframe with sample groups

In [12]:
mg_coldata <- data.frame(row.names = colnames(mg_counts), sample=colnames(mg_counts), group=mg_groups[colnames(mg_counts),"Group"],day=doy(colnames(mg_counts)))

In [13]:
mt_coldata <- data.frame(row.names = colnames(mt_counts), sample=colnames(mt_counts), group=mt_groups[colnames(mt_counts),"Group"])

For the metagenome, add day of year as a variable as well.

In [14]:
head(mg_coldata)

,sample,group,day
120314,120314,early-spring,74
120322,120322,early-spring,82
120328,120328,spring,88
120403,120403,spring,94
120416,120416,spring,107
120419,120419,spring,110


In [15]:
head(mt_coldata)

,sample,group
120712,120712,spring
120813,120813,summer
120927,120927,winter
121220,121220,winter
130123,130123,winter
130226,130226,winter


## Analysis of the MG dataset

<img style="float: left;" src="results/figures/mg_sample_clusters_groups.png",width=25%/>

Perform pair-wise comparisons of spring, early-summer, summer and winter groups.

In [17]:
groups <- c("early-spring","spring","early-summer","summer","fall","winter")
groups <- sort(groups)
groups

[1] "early-spring" "early-summer" "fall"         "spring"       "summer"      
[6] "winter"

In [18]:
done <- c()
df <- data.frame(baseMean=NA,log2FoldChange=NA,lfcSE=NA,stat=NA,pvalue=NA,padj=NA,groups="",transporter="")
for (group1 in groups){
    for (group2 in groups[!(groups%in%c(group1))]) {
        sort_groups = sort(c(group2,group1))
        x = paste(sort_groups[1],sort_groups[2],sep="X")
        if (!x%in%done){
            df_dseq = as.data.frame(dseq_groups(coldata = mg_coldata, 
                                                countdata = mg_counts, group1=group1,group2=group2))
            df <- rbind(df,df_dseq)
            done <- c(done,x)
        }
    }
}
mg_df = df[2:nrow(df),]

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
-- replacing outliers and refi

Add adjusted p-values

In [19]:
mg_df$p_total_adj = p.adjust(mg_df$pvalue,method="fdr")

In [20]:
write.table(x=mg_df,"results/mg/deseq2.tab",sep="\t",row.names = FALSE, quote=FALSE)

## Analysis of the MT dataset

<img style="float: left;" src="results/figures/mt_sample_clusters_groups.png",width=25%/>

In [21]:
groups <- c("spring","early-summer","summer","winter")
groups <- sort(groups)
groups

[1] "early-summer" "spring"       "summer"       "winter"

In [22]:
done <- c()
df <- data.frame(baseMean=NA,log2FoldChange=NA,lfcSE=NA,stat=NA,pvalue=NA,padj=NA,groups="",transporter="")
for (group1 in sort(groups)){
    for (group2 in groups[!(groups%in%c(group1))]) {
        sort_groups = sort(c(group2,group1))
        x = paste(sort_groups[1],sort_groups[2],sep="X")
        if (!x%in%done){
            df_dseq = as.data.frame(dseq_groups(coldata = mt_coldata, 
                                                countdata = mt_counts, group1=group1,group2=group2))
            df <- rbind(df,df_dseq)
            done <- c(done,x)
        }
    }
}
mt_df = df[2:nrow(df),]

converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
converting counts to integer mode
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.
final dispersion estimates
fitting mod

In [23]:
mt_df$p_total_adj = p.adjust(mt_df$pvalue,method="fdr")

In [24]:
write.table(x=mt_df,"results/mt/deseq2.tab",sep="\t",row.names = FALSE, quote=FALSE)